## A common pipeline to clean dataframes


0. Import modules
1. Normalize column names
2. Eval misspelling in object non-date columns
3. Eval & Drop duplicates without index_id `mlg.DF_wo_colX(df, colnames)`
4. Check NAs `mlg.na_absperc(df)`
    - By Columns
    - By Rows
5. Eval 'unique/freq ratio'`mlg.categ_summ(df)`
6. Eval Data types
7. Save the cleaned dataframe

## Summary of cleaning `film.csv`


**Original film.csv** had 1000 rows and 13 columns `['film_id', 'title', 'description', 'release_year', 'language_id','original_language_id', 'rental_duration', 'rental_rate', 'length','replacement_cost', 'rating', 'special_features', 'last_update']`

* 1. No column name to normalize
* 2. No misspelling values
* 3. No duplicated rows
* 4. I **droped `original_language_id`** column as it had 100% NA's
* 5. Nothing striking from unique/freq ratio analysis
* 6. No data type to transform
* 7. I splited the dataframe into two entities and saved them into separate files: <br/>
    - `clean/films.csv`
    - `clean/special_feature1.csv` 
* Extra: I review the DFs and checked there were not misspelling errors

**Cleaned film1.csv** has 1000 rows and 11 columns<br/>  `['film_id', 'title', 'description', 'release_year', 'language_id', 'rental_duration', 'rental_rate', 'length','replacement_cost', 'rating','last_update']`<br/> 

**Cleaned special_feature1.csv** has 2115 rows and 2 columns `['film_id', 'special_features']`

### 0. Import modules

First things first!!

We need to import modules and set default notebook properties as;

- leading with warnings or
- defining how we want to display the outputs and plots.

I will also import my own modulemlgfrom scr/dataanalysis1.py.
When I make modifications in my functions, I need to detatch the module and load it again!

In [12]:
# Import modules etc

import pandas as pd
import numpy as np
import re

np.random.seed(42)
pd.set_option('display.max_columns', None) # show all the columns

# print the plot in the jupyter output
%matplotlib inline 

import warnings
warnings.filterwarnings('ignore') # ignorar warnings

import pylab as plt   # import matplotlib.pyplot as plt
import seaborn as sns
import fuzzywuzzy as fzw

# Import my module
from src import dataanalysis_fun1 as mlg 

In [13]:
# Reload my module if neccessary

import importlib
from src import dataanalysis_fun1 as mlg # Import the module
#importlib.reload(mlg)  # Reload the module

# Suppress warning when reloading the module
with warnings.catch_warnings():
    warnings.simplefilter("ignore") 
    importlib.reload(mlg)  # Reload the module

In [14]:
DF_raw = pd.read_csv('../../data/raw/film.csv')

### 1. Normalize column names

In [15]:
display(DF_raw.columns)
setA = set(DF_raw.columns)
DF_raw=mlg.colnnam_clean(DF_raw)
setB = set(DF_raw.columns)

print(f'Normalized column names: {len(setB.difference(setA))}')

Index(['film_id', 'title', 'description', 'release_year', 'language_id',
       'original_language_id', 'rental_duration', 'rental_rate', 'length',
       'replacement_cost', 'rating', 'special_features', 'last_update'],
      dtype='object')

Normalized column names: 0


### 2. Eval misspelling in object non-date columns

In [16]:
for i in DF_raw.columns: 
    if (DF_raw[i].dtype == 'object') and ("date" not in i):
        print(i.upper(), ":")
        DF_raw[i]=DF_raw[i].apply(lambda a: a.strip()) ## Remove leading and trailing spaces 
        print(DF_raw[[i]].value_counts())
        print("\n")
    else:
        print(i.upper(), " ---> Non object-Column or explicit date reference")
        pass

FILM_ID  ---> Non object-Column or explicit date reference
TITLE :
title           
ACADEMY DINOSAUR    1
PERFECT GROOVE      1
PARK CITIZEN        1
PARTY KNOCK         1
PAST SUICIDES       1
                   ..
FROGMEN BREAKING    1
FRONTIER CABIN      1
FROST HEAD          1
FUGITIVE MAGUIRE    1
ZORRO ARK           1
Length: 1000, dtype: int64


DESCRIPTION :
description                                                                                                          
A Action-Packed Character Study of a Astronaut And a Explorer who must Reach a Monkey in A MySQL Convention              1
A Intrepid Tale of a Madman And a Astronaut who must Challenge a Hunter in A Monastery                                   1
A Intrepid Saga of a Man And a Forensic Psychologist who must Reach a Squirrel in A Monastery                            1
A Intrepid Saga of a Man And a Lumberjack who must Vanquish a Husband in The Outback                                     1
A Intrepid Story of a

### 3. eval/drop duplicates without index_id

In [5]:
listtest=["film_id"]
DF_raw_wo1=mlg.DF_wo_colX(DF_raw, listtest)

print("\n")

DF1=DF_raw.copy()
if any(DF_raw_wo1.duplicated()): # there are duplicates!
    print("DUPLICATED?", any(DF_raw_wo1.duplicated()))
    # Find the index positions of duplicate rows in the subset of columns
    dup_ind = DF_raw_wo1.duplicated()

    # Show the duplicate rows
    display(DF1[dup_ind])

    # Display the index positions where rows are duplicated
    nondup_ind = dup_ind[~dup_ind].index
    DF1 = DF1.iloc[nondup_ind]

    # Reset the index to have continuous index values
    DF1.reset_index(drop=True, inplace=True) 
else:
    print("NO HIDDEN DUPLICATES")
    pass


display(DF_raw.shape)
display(DF1.shape)




NO HIDDEN DUPLICATES


(1000, 13)

(1000, 13)

### 4. check NAs again

- By column
- By row

In [6]:
display(mlg.na_absperc(DF1)) # by cols
display(mlg.na_absperc(DF1.T)) # by rows

# DROP IT CAUSE IT CONTAINS 100% NAS!!
DF1=DF1.loc[:, ~DF1.columns.isin(["original_language_id"])]

,abs_NA,perc_NA
original_language_id,1000,100.0


,abs_NA,perc_NA
0,1,7.692308
671,1,7.692308
658,1,7.692308
659,1,7.692308
660,1,7.692308
...,...,...
338,1,7.692308
339,1,7.692308
340,1,7.692308
341,1,7.692308


### 5. unique/freq ratio

In [64]:
display(mlg.categ_summ(DF1).sort_values("resto_per", ascending =True))

,count,unique,top,freq,unicount_ratio,resto_abs,resto_per
last_update,1000,1,2006-02-15 05:03:42,1000,0.001,0,0.0
rating,1000,5,PG-13,223,0.005,777,77.7
special_features,1000,15,"Trailers,Commentaries,Behind the Scenes",79,0.015,921,92.1
title,1000,1000,ACADEMY DINOSAUR,1,1.0,999,99.9
description,1000,1000,A Epic Drama of a Feminist And a Mad Scientist...,1,1.0,999,99.9


### 6. data types

In [65]:
DF1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   film_id           1000 non-null   int64  
 1   title             1000 non-null   object 
 2   description       1000 non-null   object 
 3   release_year      1000 non-null   int64  
 4   language_id       1000 non-null   int64  
 5   rental_duration   1000 non-null   int64  
 6   rental_rate       1000 non-null   float64
 7   length            1000 non-null   int64  
 8   replacement_cost  1000 non-null   float64
 9   rating            1000 non-null   object 
 10  special_features  1000 non-null   object 
 11  last_update       1000 non-null   object 
dtypes: float64(2), int64(5), object(5)
memory usage: 93.9+ KB


### 7. Save the cleaned dataframe(s)

Split the dataframe into 2 entities `films1.csv`and `special_features1.csv`

In [8]:
DF2 = DF1[["film_id", "special_features"]]


display(DF2.head())
# transform the strign separated by "," into list of values
DF2['special_features'] = DF2['special_features'].str.split(',') 

# Explode the column to create multiple rows
DF2 = DF2.explode('special_features')


DF1=DF1.loc[:, ~DF1.columns.isin(["special_features"])]

,film_id,special_features
0,1,"Deleted Scenes,Behind the Scenes"
1,2,"Trailers,Deleted Scenes"
2,3,"Trailers,Deleted Scenes"
3,4,"Commentaries,Behind the Scenes"
4,5,Deleted Scenes


,film_id,special_features
0,1,"[Deleted Scenes, Behind the Scenes]"
1,2,"[Trailers, Deleted Scenes]"
2,3,"[Trailers, Deleted Scenes]"
3,4,"[Commentaries, Behind the Scenes]"
4,5,[Deleted Scenes]


,film_id,special_features
0,1,Deleted Scenes
0,1,Behind the Scenes
1,2,Trailers
1,2,Deleted Scenes
2,3,Trailers


### Extra: Review after spliting the dataframe

In [78]:
print("DUPLICATED?:", any(DF1.duplicated()), any(DF2.duplicated()))

display(DF2["special_features"].value_counts())

DF2.info()
DF2.shape

DUPLICATED?: False False


Commentaries         539
Behind the Scenes    538
Trailers             535
Deleted Scenes       503
Name: special_features, dtype: int64

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2115 entries, 0 to 999
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   film_id           2115 non-null   int64 
 1   special_features  2115 non-null   object
dtypes: int64(1), object(1)
memory usage: 49.6+ KB


(2115, 2)

In [80]:
film1=DF1.copy()
special_feature1=DF2.copy()
#film1.to_csv('../../data/clean/film1.csv', index=False)
#special_feature1.to_csv('../../data/clean/special_feature1.csv', index=False)